In [1]:
!pip install dash
!pip install dash-bootstrap-components
!pip install pandas

In [10]:
import math
from dash import Dash, dcc, html, dash_table, callback_context
from dash.dependencies import Input, Output, State
import dash_bootstrap_components as dbc
import numpy as np
import plotly.graph_objects as go
import time

app = Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

entradas = {
    "sen": "math.sin",
    "sin": "math.sin",
    "cos": "math.cos",
    "tan": "math.tan",
    "e^": "math.exp",
    "ln": "math.log",
    "log": "math.log10",
    "x^": "x**"
}

def sanitizar(sintaxis):
    for clave, valor in entradas.items():
        sintaxis = sintaxis.replace(clave, valor)
    return sintaxis

def sumaInferior(funcion, desde, hasta, rectangulos):
    if rectangulos <= 0:
        raise ValueError("En la primaria no te enseñaron que no podes dividir algo por cero?")

    if hasta <= desde:
        raise ValueError("Estas en tu casa y para ir a la Uni, vas desde la Uni a tu Casa?")
    
    a = desde
    b = hasta
    n = rectangulos
    delta = (b - a) / n
    sumatoria_I = 0.0

    for k in range(n):
        izq = a + k * delta
        der = izq + delta
        m_k = min(funcion(izq), funcion(der))
        sumatoria_I += m_k * delta

    return sumatoria_I

def sumaSuperior(funcion, desde, hasta, rectangulos):
    if rectangulos <= 0:
        raise ValueError("En la primaria no te enseñaron que no podes dividir algo por cero?")

    if hasta <= desde:
        raise ValueError("Estas en tu casa y para ir a la Uni, vas desde la Uni a tu Casa?")
    
    a = desde
    b = hasta
    n = rectangulos
    delta = (b - a) / n
    sumatoria_S = 0.0

    for k in range(n):
        izq = a + k * delta
        der = izq + delta
        M_k = max(funcion(izq), funcion(der))
        sumatoria_S += M_k * delta

    return sumatoria_S

# Variable global para almacenar el estado del cálculo
calculo_estado = {
    'en_progreso': False,
    'datos': [],
    'funcion': None,
    'desde': None,
    'hasta': None,
    'eps': None,
    'N_actual': 0,
    'completado': False,
    'N_final': 0
}

app.layout = dbc.Container([
    html.H2("Suma inferior y superior de Riemann", className="text-center my-4"),
    
    html.Div([
        dbc.Row([
            dbc.Col([
                dbc.Label("Función f(x)"),
                dbc.Input(id="funcion-input", placeholder="Ej: x**2, sin(x), e^x", type="text", value="x**2"),
            ], width=3),
            dbc.Col([
                dbc.Label("Desde (a)"),
                dbc.Input(id="desde-input", placeholder="Desde a", type="number", value=0),
            ], width=2),
            dbc.Col([
                dbc.Label("Hasta (b)"),
                dbc.Input(id="hasta-input", placeholder="Hasta b", type="number", value=1),
            ], width=2),
            dbc.Col([
                dbc.Label("Epsilon (ε)"),
                dbc.Input(id="eps-input", placeholder="Epsilon", type="number", value=0.01, step=0.001),
            ], width=2),
            dbc.Col([
                dbc.Label(" "),
                dbc.Button("Calcular", id="start-btn", color="primary", className="w-100"),
                dbc.Button("Detener", id="stop-btn", color="danger", className="w-100 mt-2", disabled=True),
            ], width=3),
        ], className="align-items-end"),
    ], className="mb-4 p-3 bg-light rounded"),
    
    dbc.Alert(id="error-alert", color="danger", is_open=False, dismissable=True),
    dbc.Alert(id="info-alert", color="info", is_open=False, dismissable=True),
    
    dbc.Progress(id="progress-bar", value=0, striped=True, animated=True, className="mb-3"),
    
    html.H4("Tabla de Resultados", className="my-3"),
    dash_table.DataTable(
        id='tabla-riemann',
        columns=[
            {"name": "N", "id": "N"},
            {"name": "Suma Inferior", "id": "SumaInferior"},
            {"name": "Integral", "id": "Integral"},
            {"name": "Suma Superior", "id": "SumaSuperior"},
            {"name": "Diferencia", "id": "Diferencia"}
        ],
        data=[],
        style_cell={'textAlign': 'center'},
        style_header={'fontWeight': 'bold'},
        # page_size=10,
        style_data_conditional=[
            {
                'if': {'row_index': 'odd'},
                'backgroundColor': 'rgb(248, 248, 248)'
            },
            {
                'if': {'column_id': 'Integral'},
                'fontWeight': 'bold',
                'color': 'darkblue'
            }
        ]
    ),
    
    html.H4("Gráfico de la Función y Rectángulos", className="my-3"),
    dcc.Graph(id="grafico-riemann"),
    
    # Componente oculto para trigger de actualizaciones
    dcc.Interval(id="interval-update", interval=500, n_intervals=0, disabled=True),
    
    # Almacenamiento para el estado del cálculo
    dcc.Store(id='calculation-store', data=calculo_estado),
], fluid=True)

@app.callback(
    Output('calculation-store', 'data'),
    Output('start-btn', 'disabled'),
    Output('stop-btn', 'disabled'),
    Output('interval-update', 'disabled'),
    Output('info-alert', 'children'),
    Output('info-alert', 'is_open'),
    Input('start-btn', 'n_clicks'),
    Input('stop-btn', 'n_clicks'),
    Input('interval-update', 'n_intervals'),
    State('funcion-input', 'value'),
    State('desde-input', 'value'),
    State('hasta-input', 'value'),
    State('eps-input', 'value'),
    State('calculation-store', 'data')
)
def control_calculo(start_clicks, stop_clicks, n_intervals, funcion_texto, desde, hasta, eps, store_data):
    ctx = callback_context
    if not ctx.triggered:
        return store_data, False, True, True, "", False
    
    trigger_id = ctx.triggered[0]['prop_id'].split('.')[0]
    
    if trigger_id == 'start-btn' and start_clicks:
        # Validar entradas
        if not funcion_texto:
            return store_data, False, True, True, "Por favor ingrese una función", True
        
        if desde is None or hasta is None:
            return store_data, False, True, True, "Por favor ingrese los límites de integración", True
        
        if eps is None or eps <= 0:
            return store_data, False, True, True, "Epsilon debe ser mayor a cero", True
        
        if hasta <= desde:
            return store_data, False, True, True, "El límite superior debe ser mayor al inferior", True
        
        try:
            # Sanitizar y crear la función
            f_python = sanitizar(funcion_texto)
            test_x = (desde + hasta) / 2
            test_result = eval(f_python, {"x": test_x, "math": math, "sin": math.sin, "cos": math.cos, "tan": math.tan, "exp": math.exp, "log": math.log, "log10": math.log10})
            
            # Inicializar estado del cálculo
            nuevo_estado = {
                'en_progreso': True,
                'datos': [],
                'funcion': funcion_texto,
                'f_python': f_python,
                'desde': desde,
                'hasta': hasta,
                'eps': eps,
                'N_actual': 1,
                'completado': False,
                'max_iteraciones': 10000,
                'N_final': 0
            }
            return nuevo_estado, True, False, False, "Cálculo iniciado...", True
            
        except Exception as e:
            return store_data, False, True, True, f"Error: {str(e)}", True
    
    elif trigger_id == 'stop-btn' and stop_clicks:
        # Detener cálculo
        store_data['en_progreso'] = False
        return store_data, False, True, True, "Cálculo detenido", True
    
    elif trigger_id == 'interval-update' and store_data.get('en_progreso', False):
        # Ejecutar un paso del cálculo
        if (store_data['N_actual'] <= store_data['max_iteraciones'] and 
            not store_data.get('completado', False)):
            
            try:
                f = lambda x: eval(store_data['f_python'], {"x": x, "math": math, "sin": math.sin, "cos": math.cos, "tan": math.tan, "exp": math.exp, "log": math.log, "log10": math.log10})
                
                inf = sumaInferior(f, store_data['desde'], store_data['hasta'], store_data['N_actual'])
                sup = sumaSuperior(f, store_data['desde'], store_data['hasta'], store_data['N_actual'])
                diferencia = abs(sup - inf)
                
                nuevo_dato = {
                    "N": store_data['N_actual'],
                    "SumaInferior": f"{inf:.6f}",
                    "Integral": f"∫{store_data['funcion']} dx",
                    "SumaSuperior": f"{sup:.6f}",
                    "Diferencia": f"{diferencia:.6f}"
                }
                
                store_data['datos'].append(nuevo_dato)
                
                if diferencia < store_data['eps']:
                    store_data['completado'] = True
                    store_data['en_progreso'] = False
                    store_data['N_final'] = store_data['N_actual']  # N final es el actual cuando se cumple la condición
                    mensaje = f"¡Cálculo completado! N final: {store_data['N_actual']}"
                    return store_data, False, True, True, mensaje, True
                else:
                    store_data['N_actual'] += 1
                    progreso = min(100, (store_data['N_actual'] / store_data['max_iteraciones']) * 100)
                    mensaje = f"Calculando... N={store_data['N_actual']-1}, Diferencia={diferencia:.6f}"
                    return store_data, True, False, False, mensaje, True
                    
            except Exception as e:
                store_data['en_progreso'] = False
                return store_data, False, True, True, f"Error en cálculo: {str(e)}", True
        
        else:
            store_data['en_progreso'] = False
            store_data['completado'] = True
            store_data['N_final'] = store_data['N_actual'] - 1  # Si llegó al límite, N_final es el último calculado
            return store_data, False, True, True, f"Cálculo alcanzó el límite de {store_data['max_iteraciones']} iteraciones", True
    
    return store_data, False, True, True, "", False

@app.callback(
    Output('tabla-riemann', 'data'),
    Output('grafico-riemann', 'figure'),
    Output('progress-bar', 'value'),
    Output('error-alert', 'children'),
    Output('error-alert', 'is_open'),
    Input('interval-update', 'n_intervals'),
    State('calculation-store', 'data')
)
def actualizar_ui(n_intervals, store_data):
    if not store_data.get('datos'):
        return [], go.Figure(), 0, "", False
    
    datos = store_data['datos']
    
    # Calcular progreso
    if store_data.get('completado', False):
        progreso = 100
    else:
        progreso = min(100, (store_data['N_actual'] / store_data['max_iteraciones']) * 100)
    
    # Crear gráfico - SIEMPRE que haya datos completados
    fig = go.Figure()
    
    if datos and store_data.get('completado', False):
        try:
            f = lambda x: eval(store_data['f_python'], {"x": x, "math": math, "sin": math.sin, "cos": math.cos, "tan": math.tan, "exp": math.exp, "log": math.log, "log10": math.log10})
            
            # Graficar función
            x_cont = np.linspace(store_data['desde'], store_data['hasta'], 500)
            y_cont = [f(x) for x in x_cont]
            fig.add_trace(go.Scatter(
                x=x_cont, y=y_cont, mode='lines', 
                name=f'f(x) = {store_data["funcion"]}',
                line=dict(color='black', width=2)
            ))
            
            # Agregar rectángulos - SIN LÍMITE pero optimizado para no saturar
            N_final = store_data.get('N_final', 0)
            if N_final > 0:
                dx = (store_data['hasta'] - store_data['desde']) / N_final
                
                # Para N grandes, mostrar solo una muestra de rectángulos
                if N_final > 100:
                    # Mostrar máximo 100 rectángulos espaciados uniformemente
                    step = max(1, N_final // 100)
                    indices = range(0, N_final, step)
                else:
                    # Para N pequeños, mostrar todos
                    indices = range(N_final)
                
                x_rects = [store_data['desde'] + k * dx for k in range(N_final)]
                
                for i in indices:
                    x0 = x_rects[i]
                    x1 = x0 + dx
                    h_inf = min(f(x0), f(x1))
                    h_sup = max(f(x0), f(x1))
                    
                    # Rectángulo inferior
                    fig.add_trace(go.Scatter(
                        x=[x0, x0, x1, x1, x0], y=[0, h_inf, h_inf, 0, 0],
                        fill="toself", fillcolor="blue", line=dict(color="blue"),
                        opacity=0.3, showlegend=(i == indices[0]), name="Suma Inferior"
                    ))
                    
                    # Rectángulo superior
                    fig.add_trace(go.Scatter(
                        x=[x0, x0, x1, x1, x0], y=[0, h_sup, h_sup, 0, 0],
                        fill="toself", fillcolor="red", line=dict(color="red"),
                        opacity=0.3, showlegend=(i == indices[0]), name="Suma Superior"
                    ))
            
            fig.update_layout(
                title=f"Suma inferior (azul) y superior (roja) de Riemann - N={N_final}",
                xaxis_title="x", yaxis_title="f(x)", showlegend=True, height=500
            )
            
        except Exception as e:
            return datos, fig, progreso, f"Error al crear gráfico: {str(e)}", True
    
    return datos, fig, progreso, "", False

if __name__ == "__main__":
    app.run(debug=True, port=8099)